## Session 1
Define a function that returns a list of numbers that sum up to one.

In [1]:
from random import random
from numpy import zeros
#returns a list of n numbers that aproximately sum up to one
def exercise1(n):
    if (n<=0): raise Exception('The given number must be a positive integer.')
    if (n==1): return [1]
    numeros=zeros(n)
    numeros[0:n-1]=[random() for i in range(n-1)]
    numeros[n-1]=-sum(numeros[0:n-1])+1
    return numeros

In [2]:
x=exercise1(5)
print('List: '+str(x))
print('Sum: '+str(sum(x)))

List: [ 0.90576161  0.7443093   0.52087507  0.34304289 -1.51398887]
Sum: 1.0


In [3]:
from random import random,randint
#returns a list of random size, within the specified range of positive numbers, of numbers that aproximately sum up to one
def exercise1(inf,sup):
    if (inf<=0): raise Exception('The given range must be positive.')
    n=randint(inf,sup)
    numeros=[random() for i in range(n)]
    return [numeros[i]/sum(numeros) for i in range(n)]

In [4]:
x=exercise1(1,1000)
print('Number of elements in the list: '+str(len(x)))
print('Sum: '+str(sum(x)))

Number of elements in the list: 108
Sum: 0.9999999999999997


## Session 2
1. Create a 10x10 2d array with 1 on the border and 0 inside.

In [6]:
import numpy as np
Z=np.zeros((10,10))
Z[0:10,(0,-1)]=np.ones((10,2))
Z[(0,-1),0:10]=np.ones((2,10))
print(Z)

[[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]]


2. Create a structured array representing a position (x,y) and a color (r,g,b)

In [7]:
Z = np.zeros(10, [ ('position', [ ('x', float, 1),
                                   ('y', float, 1)]),
                    ('color',    [ ('r', float, 1),
                                   ('g', float, 1),
                                   ('b', float, 1)])])
print(Z)

[(( 0.,  0.), ( 0.,  0.,  0.)) (( 0.,  0.), ( 0.,  0.,  0.))
 (( 0.,  0.), ( 0.,  0.,  0.)) (( 0.,  0.), ( 0.,  0.,  0.))
 (( 0.,  0.), ( 0.,  0.,  0.)) (( 0.,  0.), ( 0.,  0.,  0.))
 (( 0.,  0.), ( 0.,  0.,  0.)) (( 0.,  0.), ( 0.,  0.,  0.))
 (( 0.,  0.), ( 0.,  0.,  0.)) (( 0.,  0.), ( 0.,  0.,  0.))]


3. Consider a large vector Z, compute Z to the power of 3 using 2 different methods

In [8]:
x = np.random.rand(int(5e7))

%timeit np.power(x,3) #1.85s
%timeit x*x*x #360ms
%timeit for i in range(len(x)): x[i]**3 #15.4s

1 loop, best of 3: 1.74 s per loop
1 loop, best of 3: 357 ms per loop
1 loop, best of 3: 14.3 s per loop


## Session 4
### Challenge 1: Fix the script!
The script was disorganized; many objects were called before being defined. I fixed it by organizing it according to the VTK pipeline. Once everything was in place the following result was displayed:
<img src="challenge1sess4.png">

In [9]:
import vtk

In [10]:
pl3d = vtk.vtkMultiBlockPLOT3DReader()
xyx_file = "data/combxyz.bin"
q_file = "data/combq.bin"
pl3d.SetXYZFileName(xyx_file)
pl3d.SetQFileName(q_file)
pl3d.SetScalarFunctionNumber(100)
pl3d.SetVectorFunctionNumber(202)
pl3d.Update()

In [11]:
blocks = pl3d.GetOutput()
b0 = blocks.GetBlock(0)
points = vtk.vtkVertexGlyphFilter()
points.SetInputData(b0)
outline = vtk.vtkStructuredGridOutlineFilter()
outline.SetInputData(b0)

#mappers
outlineMapper = vtk.vtkPolyDataMapper()
outlineMapper.SetInputConnection(outline.GetOutputPort())

scalarsMapper = vtk.vtkPolyDataMapper()
scalarsMapper.SetInputConnection(points.GetOutputPort())
scalarsMapper.SetScalarModeToUsePointData()

#actors
outlineActor = vtk.vtkActor()
outlineActor.SetMapper(outlineMapper)
outlineActor.GetProperty().SetColor(0.6,0.5,0.9)

scalarsActor = vtk.vtkActor()
scalarsActor.SetMapper(scalarsMapper)

#rendering
renderer = vtk.vtkRenderer()
renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetInteractorStyle(vtk.vtkInteractorStyleTrackballCamera())
interactor.SetRenderWindow(renderWindow)
renderWindow.SetInteractor(interactor)

renderer.SetBackground(0.6,0.5,0.9)
renderer.AddActor(scalarsActor)
renderer.AddActor(outlineActor)
interactor.Start()

### Homework
Using the code from this lab session, create a Python script that contains all the steps (filters, mappers, actors,...)  that we discussed.

I created the script to visualize the velocity vector of a subset of the jet dataset as arrows. The arrows point in the same direction as the vectors and are colored proportionally to their magnitude.

<img src="sess4homework.png">

In [13]:
import vtk
#help(vtk.vtkRectilinearGridReader())

rectGridReader = vtk.vtkRectilinearGridReader()
rectGridReader.SetFileName("C:/users/bananin/documents/scientific computing/week 1/data/jet4_0.500.vtk")
# do not forget to call "Update()" at the end of the reader
rectGridReader.Update()
rectGridOutline = vtk.vtkRectilinearGridOutlineFilter()
rectGridOutline.SetInputData(rectGridReader.GetOutput())

magnitudeCalcFilter = vtk.vtkArrayCalculator()
magnitudeCalcFilter.SetInputConnection(rectGridReader.GetOutputPort())
magnitudeCalcFilter.AddVectorArrayName('vectors')
# Set up here the array that is going to be used for the computation ('vectors')
magnitudeCalcFilter.SetResultArrayName('magnitude')
# Set up here the function that calculates the magnitude of a vector
magnitudeCalcFilter.SetFunction("mag(vectors)")
magnitudeCalcFilter.Update()

#Extract the data from the result of the vtkCalculator
points = vtk.vtkPoints()
grid = magnitudeCalcFilter.GetOutput()
grid.GetPoints(points)
scalars = grid.GetPointData().GetArray('magnitude')
    
#There are too many points, let's filter the points
subset = vtk.vtkMaskPoints()
subset.SetOnRatio(6000)
#subset.RandomModeOn()
subset.SetInputData(grid)

arrow=vtk.vtkArrowSource()
#Make a vtkPolyData with a vertex on each point.
#pointsGlyph = vtk.vtkVertexGlyphFilter()
glyph=vtk.vtkGlyph3D()
glyph.SetInputConnection(subset.GetOutputPort())
glyph.SetSourceConnection(arrow.GetOutputPort())
glyph.SetVectorModeToUseVector()
glyph.SetColorModeToColorByScalar()
glyph.SetScaleModeToDataScalingOff()
glyph.OrientOn()
glyph.SetScaleFactor(0.2)
#pointsGlyph.SetInputConnection(subset.GetOutputPort())
#pointsGlyph.SetInputData(ugrid)
#pointsGlyph.Update()
glyph.Update()

#mappers
glyphMapper=vtk.vtkPolyDataMapper()
glyphMapper.SetInputConnection(glyph.GetOutputPort())
glyphMapper.ScalarVisibilityOn()
#pointsMapper = vtk.vtkPolyDataMapper()
#pointsMapper.SetInputConnection(pointsGlyph.GetOutputPort())
#pointsMapper.SetScalarModeToUsePointData()
rectGridOutlineMapper = vtk.vtkPolyDataMapper()
rectGridOutlineMapper.SetInputConnection(rectGridOutline.GetOutputPort())

#actors
pointsActor = vtk.vtkActor()
pointsActor.SetMapper(glyphMapper)
outlineActor = vtk.vtkActor()
outlineActor.SetMapper(rectGridOutlineMapper)
outlineActor.GetProperty().SetColor(0.1, 0.1, 0.1)

# Render of the filtered points
renderer = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(renderer)
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)
 
renderer.AddActor(pointsActor)
renderer.AddActor(outlineActor)
renderer.SetBackground(238/255,130/255,238/255)
renderer.ResetCamera()
renderer.GetActiveCamera().Elevation(60.0)
renderer.GetActiveCamera().Azimuth(30.0)
renderer.GetActiveCamera().Zoom(1.0)
 
renWin.SetSize(300, 300)
 
# interact with data
renWin.Render()
iren.Start()